In [1]:
import logging

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import warnings
import os

# Squash warning messages for cleaner output in the notebook
warnings.showwarning = lambda *args, **kwargs: None

import azureml.core
from azureml.core import Experiment, Workspace, Dataset
from azureml.train.automl import AutoMLConfig
from datetime import datetime
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

ModuleNotFoundError: No module named 'azureml.core'

In [ ]:
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

In [ ]:
ws = Workspace.from_config()
experiment_name = 'automl-forecasting-energydemand'
experiment = Experiment(ws, experiment_name)
amlcompute_cluster_name = "olt-auto-ml"
compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
dataset_name = 'energy-train'
energy_ds = Dataset.get_by_name(workspace=ws, name=dataset_name)


In [ ]:
from azureml.automl.core.featurization.featurizationconfig import FeaturizationConfig

featurization_config = FeaturizationConfig()

# We remove constant features.
featurization_config.drop_columns = ['TZ', 'City', 'Zone']

# Fill mising values in the `Energy` column with forward fill value.
featurization_config.add_transformer_params('Imputer', ['Load'], {"strategy": "ffill"})

In [ ]:
from azureml.automl.core.forecasting_parameters import ForecastingParameters
forecasting_parameters = ForecastingParameters(
    time_column_name='Date',
    forecast_horizon=48,
    target_rolling_window_size=3,
    feature_lags='auto',
    country_or_region_for_holidays='US',
    validate_parameters=True
)

In [ ]:
automl_config = AutoMLConfig(task='forecasting',
                             primary_metric='r2_score',
                             training_data=energy_ds,
                             label_column_name='Load',
                             compute_target=compute_target,
                             featurization=featurization_config,
                             enable_dnn=False,
                             enable_ensembling=True,
                             enable_early_stopping=True,
                             experiment_timeout_minutes=15,
                             blocked_models = ['Prophet'],
                             n_cross_validations=5,
                             forecasting_parameters=forecasting_parameters)

In [ ]:
remote_run = experiment.submit(automl_config, show_output=False)
remote_run

In [ ]:
remote_run.wait_for_completion()
best_run, fitted_model = remote_run.get_output()

In [ ]:
fitted_model.named_steps['timeseriestransformer'].get_engineered_feature_names()

In [ ]:
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

In [ ]:
print_model(fitted_model)